In [2]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [3]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET","/summary",payload,headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

In [5]:
covid1 = json.loads(data)

In [6]:
pd.json_normalize(covid1['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,17,39285,0,1458,53,32842,2020-10-03T01:53:38Z
1,Albania,AL,albania,157,13806,1,388,230,8077,2020-10-03T01:53:38Z
2,Algeria,DZ,algeria,160,51690,5,1741,108,36282,2020-10-03T01:53:38Z
3,Andorra,AD,andorra,0,2050,0,53,0,1432,2020-10-03T01:53:38Z
4,Angola,AO,angola,142,5114,2,185,141,2082,2020-10-03T01:53:38Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,1,1095,0,35,0,1010,2020-10-03T01:53:38Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-03T01:53:38Z
185,Yemen,YE,yemen,5,2039,0,587,11,1297,2020-10-03T01:53:38Z
186,Zambia,ZM,zambia,43,14802,1,333,2,13961,2020-10-03T01:53:38Z


In [7]:
df = pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,17,39285,0,1458,53,32842,2020-10-03T01:53:38Z,{}
1,Albania,AL,albania,157,13806,1,388,230,8077,2020-10-03T01:53:38Z,{}
2,Algeria,DZ,algeria,160,51690,5,1741,108,36282,2020-10-03T01:53:38Z,{}
3,Andorra,AD,andorra,0,2050,0,53,0,1432,2020-10-03T01:53:38Z,{}
4,Angola,AO,angola,142,5114,2,185,141,2082,2020-10-03T01:53:38Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,1,1095,0,35,0,1010,2020-10-03T01:53:38Z,{}
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-03T01:53:38Z,{}
185,Yemen,YE,yemen,5,2039,0,587,11,1297,2020-10-03T01:53:38Z,{}
186,Zambia,ZM,zambia,43,14802,1,333,2,13961,2020-10-03T01:53:38Z,{}


In [9]:
covid2 = df.drop(columns=['CountryCode','Slug','Date','Premium'],axis = 1)
covid2

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,17,39285,0,1458,53,32842
1,Albania,157,13806,1,388,230,8077
2,Algeria,160,51690,5,1741,108,36282
3,Andorra,0,2050,0,53,0,1432
4,Angola,142,5114,2,185,141,2082
...,...,...,...,...,...,...,...
183,Viet Nam,1,1095,0,35,0,1010
184,Western Sahara,0,10,0,1,0,8
185,Yemen,5,2039,0,587,11,1297
186,Zambia,43,14802,1,333,2,13961


In [10]:
m = folium.Map(tiles = 'Stamen Terrain',min_zoom = 1.5)
m

In [11]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'


In [16]:
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom = 2,
    name = 'Covid-19',
    data = covid2,
    columns = ['Country','TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    nama_fill_color = 'black',
    legend_name = 'Total Confirmed Covid Cases',
).add_to(m)
m

In [17]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [18]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [19]:
covid_final = pd.merge(covid2,coordinates,on='Country')

In [20]:
def PlotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radus = 5,
                       weight = 2,
                       popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                       fill_color = '#000000').add_to(m)

In [22]:
covid_final.apply(PlotDot,axis = 1)
m.fit_bounds(m.get_bounds())
m

In [23]:
ml = folium.Map(tiles = 'StamenToner',min_zoon = 2)
ml

In [24]:
deaths = covid_final['TotalDeaths'].astype(float)

In [26]:
lat = covid_final['latitude'].astype(float)

In [27]:
lon = covid_final['longitude'].astype(float)

In [28]:
ml.add_child(HeatMap(zip(lat,lon,deaths),radius = 0))

In [29]:
def PlotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radus = 5,
                       weight = 2,
                       popup = [point.Country,point.TotalDeaths],
                       fill_color = '#000000').add_to(ml)
covid_final.apply(PlotDot,axis = 1)
ml.fit_bounds(ml.get_bounds())
ml